In [1]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
import pandas as pd
import copy
import math
import random as rnd
import time
%matplotlib inline

In [2]:
start_time = time.time()
#defining problem's variable
chessboard_size = 100
#initial temperature
T0 = 500
#geometrical temperature constant
geom = 0.95
#controlling variable for number of iteration per temperature changes.
a = 0.1
#b variable will be the same as chessboard_size
#b = 50

In [3]:
def fitnesscheck(chessboard):
    fitness = 0
    x = np.arange(chessboard_size)
    y = []
    for k in range(chessboard_size):
        y.append(chessboard[k,:].tolist().index(0))
    y = np.array(y)
    for i in range(chessboard_size):
        #check 4 diagonal
        #diagonal tenggara
        for j in range(chessboard_size-1):
            try:
                if chessboard[x[i]+(j+1)][y[i]+(j+1)] == float(0):
                    if x[i]+(j+1) >= 0 and y[i]+(j+1) >= 0:
                        #print('tenggara, queen ' + str(x[i]) + ',' +str(y[i]) +' found enemy in ' + str([x[i]+(j+1)]) + ',' + str([y[i]+(j+1)]))
                        fitness = fitness + 1
            except IndexError:
                break
        #diagonal barat daya
        for j in range(chessboard_size-1):
            try:
                if chessboard[x[i]+(j+1)][y[i]-(j+1)] == float(0):
                    if x[i]+(j+1) >=0 and y[i]-(j+1) >= 0:
                        #print('barat daya, queen ' + str(x[i]) + ',' +str(y[i]) +' found enemy in ' + str([x[i]+(j+1)]) + ',' + str([y[i]-(j+1)]))
                        fitness = fitness + 1
            except IndexError:
                break
        #diagonal timur laut
        for j in range(chessboard_size-1):
            try:
                if chessboard[x[i]-(j+1)][y[i]+(j+1)] == float(0):
                    if x[i]-(j+1) >= 0 and y[i]+(j+1) >= 0:
                        #print('timur laut, queen ' + str(x[i]) + ',' +str(y[i]) +' found enemy in ' + str([x[i]-(j+1)]) + ',' + str([y[i]+(j+1)]))
                        fitness = fitness + 1
            except IndexError:
                break
        #diagonal barat laut
        for j in range(chessboard_size-1):
            try:
                if chessboard[x[i]-(j+1)][y[i]-(j+1)] == float(0):
                    if x[i]-(j+1) >= 0 and y[i]-(j+1) >= 0:
                        #print('barat daya, queen ' + str(x[i]) + ',' +str(y[i]) +' found enemy in ' + str([x[i]-(j+1)]) + ',' + str([y[i]-(j+1)]))
                        fitness = fitness + 1
            except IndexError:
                break
    return fitness

In [4]:
def create_chessboard():
    #create chessboard size
    chessboard = np.full((chessboard_size**2), 0.5)
    #create checkered board
    if chessboard_size % 2 != 0:
        chessboard[::2] = 1
        chessboard = chessboard.reshape((chessboard_size, chessboard_size))
    else:
        chessboard = chessboard.reshape((chessboard_size, chessboard_size))
        for i in range(len(chessboard)):
            if (i % 2) != 0:
                chessboard[i][1::2] = 1
            else:
                chessboard[i][::2] = 1
    return chessboard

In [5]:
def random_placing(chessboard):
    #create random placing for queens
    x = np.random.permutation(chessboard_size)
    y = np.random.permutation(chessboard_size)
    for i in range(len(chessboard)):
        chessboard[x[i]][y[i]] = 0
    return chessboard

In [6]:
#change queen position with other queen
#for example if there is queen in A8 and B7, they will change places become A7 and B8
def change_position(chessboard):
    reset = create_chessboard()
    queen_number = np.arange(chessboard_size)
    random1 = rnd.choice(queen_number)
    random2 = rnd.choice(queen_number)
    a = chessboard[random1].tolist().index(0)
    b = chessboard[:,random2].tolist().index(0)
    chessboard[random1][a], chessboard[random1][random2] = chessboard[random1][random2], chessboard[random1][a]
    chessboard[b][random2], chessboard[b][a] = chessboard [b][a], chessboard[b][random2]
    if a != random2:
        chessboard[random1][a] = copy.deepcopy(reset[random1][a])
        chessboard[b][random2] = copy.deepcopy(reset[b][random2])
    return chessboard

In [7]:
#calc difference from last best fitness
def fitness_calc(fitness, fitness1):
    diff = fitness1 - fitness
    return diff

In [8]:
#defining Travelling Salesman problem

#create a number of iteration per temperature
def iter_temp(k):
    t = T0*(geom)**k
    n_per_temp = round((a*math.fabs(t-T0))) + chessboard_size
    return t, n_per_temp

#calc difference from last best distance to the distance created now
def dist_calc(distance, distance1):
    diff = distance1 - distance
    return diff


In [9]:
#create full loop of Simulated Annealing
def Simulated_Annealing():
    #cities_x, cities_y = createcities()
    #cities_df = pd.DataFrame({'x' :cities_x, 'y':cities_y})
    
    #create random queen placing by calling create_chessboard and random placing function
    chessboard = random_placing(create_chessboard())
    #calculate fitness using fitness check function
    fitness = fitnesscheck(chessboard)
    chessboard_list = [chessboard.tolist()]
    fitness_list = [fitness]
    #variable of iteration to change the temperature after number of iteration (n_per_temp) is done in temperature (t)
    k = 0
    t = T0
    #creating loop while until the temperature is lowered to 0.1
    while t > 0.1:
        #recalculating temperature with changed k variable to create new t and its n_per_temp
        t, n_per_temp = iter_temp(k)
        #run iteration for n_per_temp time
        for i in range(n_per_temp):
            #exchange the queen position by using change_position function
            change_pos1 = copy.deepcopy(chessboard)
            change_pos2 = change_position(change_pos1)
            #recalculate the fitness after changing queen position
            fitness1 = fitnesscheck(change_pos2)
            #find the fitness difference and then using logical operator to choose whether move or not
            diff = fitness_calc(fitness, fitness1)
            if diff > 0:
                move_prob = math.exp(-diff/t)
                rnd_number = rnd.random()
                if rnd_number < move_prob:
                    chessboard_list.append(change_pos1.tolist())
                    chessboard = copy.deepcopy(change_pos1)
                    fitness_list.append(fitness1)
                    fitness = copy.deepcopy(fitness1)
            else:
                chessboard_list.append(change_pos1.tolist())
                chessboard = copy.deepcopy(change_pos1)
                fitness_list.append(fitness1)
                fitness = copy.deepcopy(fitness1)
        k = k+1
    return chessboard, fitness, chessboard_list, fitness_list

In [10]:
#create full loop of Simulated Annealing
def Simulated_Annealing2():
    #cities_x, cities_y = createcities()
    #cities_df = pd.DataFrame({'x' :cities_x, 'y':cities_y})
    ###chessboard_list2 = []
    #iteration convergence check
    l = 0
    iter_check = 0
    last_iter_list = []
    #creating loop while until the temperature is lowered to 0.1
    while iter_check < 6:
        #create random queen placing by calling create_chessboard and random placing function
        chessboard = random_placing(create_chessboard())
        #calculate fitness using fitness check function
        fitness = fitnesscheck(chessboard)
        #variable of iteration to change the temperature after number of iteration (n_per_temp) is done in temperature (t)
        k = 0
        t = T0
        ###chessboard_list = [chessboard.tolist()]
        while t > 0.1:
            #recalculating temperature with changed k variable to create new t and its n_per_temp
            t, n_per_temp = iter_temp(k)
            #run iteration for n_per_temp time
            for i in range(n_per_temp):
                #exchange the queen position by using change_position function
                change_pos1 = copy.deepcopy(chessboard)
                change_pos2 = change_position(change_pos1)
                #recalculate the fitness after changing queen position
                fitness1 = fitnesscheck(change_pos2)
                #find the fitness difference and then using logical operator to choose whether move or not
                diff = fitness_calc(fitness, fitness1)
                if diff > 0:
                    move_prob = math.exp(-diff/t)
                    rnd_number = rnd.random()
                    if rnd_number < move_prob:
                        ###chessboard_list.append(change_pos2.tolist())
                        chessboard = copy.deepcopy(change_pos2)
                        fitness = copy.deepcopy(fitness1)
                else:
                    ###chessboard_list.append(change_pos2.tolist())
                    chessboard = copy.deepcopy(change_pos2)
                    fitness = copy.deepcopy(fitness1)
            k = k+1
        if len(last_iter_list) == 0:
            last_iter_list.append(change_pos2.tolist())
            l = l + 1
        else:
            x = []
            for i in range(len(last_iter_list)):
                x.append(last_iter_list[i] == change_pos2.tolist())
            if all(x) == False:
                last_iter_list.append(change_pos2.tolist())
                l = l + 1
            else:
                iter_check = iter_check + 1
        ###chessboard_list2.append(chessboard_list)     
    return l, last_iter_list

In [ ]:
#Calling the Main function code to run the Simulated Annealing program and returning the list of it's route from first
#to last iteration
x, y, z, aa = Simulated_Annealing()

In [ ]:
x

In [ ]:
y

In [ ]:
aa_array = np.array(aa)

In [ ]:
aa_array = aa_array / 2

In [ ]:
aa = aa_array.tolist()

In [ ]:
aa_df = pd.DataFrame(aa, columns = ['Value'])

In [ ]:
#Calling the Main function code to run the Simulated Annealing program and returning the list of it's route from first
#to last iteration
#x, y = Simulated_Annealing2()

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#Creating plot for last Simulated Annealing
fig = plt.figure(figsize = (20,20))
ax = fig.add_subplot(111)
ax.matshow(z[-1], cmap = "RdGy")
x = np.arange(chessboard_size)
y = []
for k in range(chessboard_size):
    y.append(np.array(z[-1])[k,:].tolist().index(0))
y = np.array(y)
for i in range(len(z[-1])):
    ax.text(y[i] ,x[i], s=u"\u2655", fontsize = 50, color = 'white', va='center', ha='center')

In [ ]:
#Creating Simulated Annealing Animation
fig = plt.figure(figsize = (20,20))
ax = fig.add_subplot(221)
#ax.set_xlim((0, 60))
#ax.set_ylim((0, 60))
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
queen = ax.matshow(z[0], cmap="RdGy")

ax = fig.add_subplot(222)
ax.set_xlim((0, 7000))
ax.set_ylim((0, 30))
#ax.axes.get_xaxis().set_visible(False)
#ax.axes.get_yaxis().set_visible(False)
convergence, = ax.plot([] , [], color = 'red', lw = 2)
'''x = np.arange(chessboard_size)
y = []
for k in range(chessboard_size):
    y.append(np.array(z[0])[k,:].tolist().index(0))
y = np.array(y)
set_queens = [ax.text(y[i] ,x[i] , s=u"\u2655", fontsize = 50, color = 'white', va='center', ha='center') for i in range(len(z[0]))]
'''

In [ ]:
def update(i):
    '''x = np.arange(chessboard_size)
    y = []
    for k in range(chessboard_size):
        y.append(np.array(z[i])[k,:].tolist().index(0))
    y = np.array(y)'''
    queen.set_data(z[i])
    convergence.set_data(aa_df.index[i], aa_df['Value'][i])
    #set_queens = [ax.text(y[j] ,x[j] , s=u"\u2655", fontsize = 50, color = 'white', va='center', ha='center') for j in range(len(z[0]))]
    return queen, convergence#, set_queens

#because there are so many frame, I reduced the frame to be animated to around 400 frame
frame_step = len(z)//200
ani = animation.FuncAnimation(fig, update, frames = range(0, len(z), frame_step), interval=100, blit = False)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#ani.save('./animation.gif', writer='imagemagick', fps=10)